<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/niche_domain_finder/niche_domain_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Script by @LeeFootSEO
**Early Access To Apps & Feature Requests:** https://www.patreon.com/leefootseo

**Buy me a Coffee:** https://www.buymeacoffee.com/leefootseo

# Niche Finder Pro
by https://twitter.com/LeeFootSEO 10/06/2021 <- follow for more SEO Python Scripts.


1.   Check if Domain is Available to Register (Free)
2.   Appraise Domain Value (GoDaddy feature based on similar sites) (Free)
3.   Check Keyword Slope (Free)
4.   Scrape SERPs for Competiton (# of sites with exact KW in the title) (Paid)


In [ ]:
!pip install pytrends
!pip install pandas
!pip install fabric3

In [ ]:
import glob
import json
import os
import io
from google.colab import files 
import pandas as pd
import requests

from datetime import datetime, time
import time as t
from datetime import datetime
from random import randint
from time import sleep
from dateutil.relativedelta import relativedelta
from pytrends.request import TrendReq
from fabric.colors import green, red, magenta  # pip install fabric3

# Enter TLDs to check and bad words to block below.

In [ ]:
bad_words = ["free", "download", "pdf"]  # enter bad words to filter
extensions = ["com", "co.uk"]  #enter domain combinations to check

#Enter Your API Details Below

1.   GoDaddy API (Free) - https://developer.godaddy.com/getstarted
2.   ZENSerp API (Paid) - https://zenserp.com/

In [ ]:
# godaddy developer key and secret (Mandatory)
api_key = "yourgodaddyapihere"   # Your Key Here
secret_key = "yourgodaddysecrethere"  # Your Key Here

# zenserp key (paid) (Optional)
zenserp_key = "yourzenserpkeyhere"  # Your Key Here

In [ ]:
# Set GoDaddy API Delays (2-3 seconds is the sweet spot for GoDaddy!)
domain_check_speed = 2  # godaddy API
appraisal_check_speed = 4  # godaddy API

# set godaddy vars
chunk_size = 500  # number of domains to check in each call.
max_length = 30  # filter domain names by length
min_price = 0  # filter domain names by price range
max_price = 5000  # filter domain names by price range
min_appr_price = 0  # if appraisal is enabled, only include domain names with min appraisal price
min_sale_price = 0  # when a domain is appraised, Godaddy API returns similar domains sold.
min_sale_year = 2000  # when a domain is appraised, Godaddy API returns similar domains sold.

In [ ]:
# create empty lists / dictionaries to hold the data
prefixes = []
keywords = []
suffixes = []
all_domains = []
similar_domains = []
found_domains = {}
domain_value = []
# Domain availability and appraisal end points
url = "https://api.godaddy.com/v1/domains/available"
appraisal = "https://api.godaddy.com/v1/appraisal/{}"
headers = {"Authorization": "sso-key {}:{}".format(api_key, secret_key)}  # godaddy api key is sent in the header
pytrend = TrendReq()

# Import Keywords from Ahrefs Export

In [ ]:
uploaded = files.upload()
df_keyword_list = list(uploaded.keys())[0]  # get the filename from the upload

In [ ]:
# read in the ahrefs export, massage the data to check domain availability.
df_keywords = pd.read_csv(df_keyword_list)
df_keywords['domain_keywords'] = df_keywords['Keyword']
df_keywords['domain_keywords'] = df_keywords['domain_keywords'].apply(lambda x: x.replace(" ", ""))
keywords = list(df_keywords['domain_keywords'])
kw_len = len(keywords)
df_keywords = df_keywords.sort_values(by="Keyword", ascending=True)
df_keywords

In [ ]:
# Generate domains
for keyword in keywords:
    for extension in extensions:
        domain = "{}.{}".format(keyword, extension)
        # Filter by length
        if len(domain) <= max_length:
            all_domains.append(domain)
all_domains

In [ ]:
# Remove blacklisted words
df_all_domains = pd.DataFrame(all_domains, columns=['Keyword'])
kws_before = (len(df_all_domains))
bad_words = "|".join(bad_words)  # changes list into | separated string for matching
df_all_domains = (df_all_domains[~df_all_domains.Keyword.str.contains(bad_words)]) #drops partial match words
kws_after = (len(df_all_domains))
all_domains = list(df_all_domains['Keyword'])  #changes df back into list
kw_diff = kws_before - kws_after
print("Removed", kw_diff, "Badwords!")
df_all_domains

In [ ]:
# This function splits all domains into chunks of a given size
def chunks(array, size):
    for i in range(0, len(array), size):
        yield array[i:i + size]

# Split the original array into subarrays
domain_chunks = list(chunks(all_domains, chunk_size))
print("--------------------------------------------------------")
print(magenta("Checking which domains are available to register"))
print("--------------------------------------------------------")
# For each domain chunk (ex. 500 domains)
for domains in domain_chunks:
    # Get availability information by calling availability API
    availability_res = requests.post(url, json=domains, headers=headers)
    # Get only available domains with price range
    for domain in json.loads(availability_res.text)["domains"]:
        if domain["available"]:
            price = float(domain["price"]) / 1000000
            if price >= min_price and price <= max_price:
                print("{:30} : ${:10}".format(domain["domain"], price))
                found_domains[domain["domain"]] = price

    # API call frequency should be ~ 30 calls per minute
    t.sleep(domain_check_speed)

# make dataframe from dictionary of available domains to register
df_found_domains = pd.DataFrame.from_dict(found_domains, orient="index")
df_found_domains = df_found_domains.reset_index()
df_found_domains

df_found_domains.columns = ['Keyword', "Price"]
  
# output all available domains to register
df_found_domains.to_csv('all_available_domains_to_register.csv')

files.download('all_available_domains_to_register.csv')
found_domains_len = len(found_domains)
print("")
print(green((found_domains_len)) + green((" of ")) + green((kw_len)) + green((" domains are available to register!")))
print("")
print("--------------------------------------------------------")
print(red("Exported all_available_domains_to_register.csv!"))

# Appraise the Value of the Domains - Free Via GoDaddy API
Optional Domain Appraisal based on similar domains sold in the past.

In [ ]:
print("--------------------------------------------------------")
print(magenta("Appraising domain values based on similar sales"))
print("--------------------------------------------------------")

for domain, price in found_domains.items():
    # Call appraisal API
    try:
        appraisal_res = requests.get(appraisal.format(domain), headers=headers).json()
    except Exception:
        print(red("Error Getting Data!"))
        govalue = 100
        pass
    try:
        # Get appraisal and similar sold domains
        govalue = appraisal_res["govalue"]
        comparable_sales = appraisal_res["comparable_sales"]

    except:
        # todo debug print
        print(appraisal_res)
        print(red("Error Getting Data! - line 192"))
        domain_value.append("Error Getting Data!")
        continue

    # Filter by min appraisal price
    if govalue >= min_appr_price:
        domain_value.append(govalue)
        print("{:30} : ${:10} : ${}".format(domain, price, govalue))
    for sale in comparable_sales:
        # Filter similar sold domains by price and year
        if sale["price"] >= min_sale_price and sale["year"] >= min_sale_year:
            similar_domain = "{:30} : {:10} : {:10}".format(
                sale["domain"], sale["price"], sale["year"])
            # Do not include duplicates
            if similar_domain not in similar_domains:
                similar_domains.append(similar_domain)

    # Do not abuse the API!!!
    t.sleep(appraisal_check_speed)

df_found_domains["Appraised Value"] = domain_value
df_found_domains['temp_match'] = df_found_domains['Keyword'].str.split('.').str[0]

df_final = pd.merge(df_found_domains, df_keywords, left_on="temp_match", right_on="domain_keywords", how="left")

# clean up and refine the data
cols = "Keyword_y", "Keyword_x", "Volume", "Global volume", "Price", "Appraised Value", "Country", "CPC", "Clicks", "Return Rate"
df_final = df_final.reindex(columns=cols)
df_final.rename(columns={"Keyword_y": "Keyword", "Keyword_x": "Domain Name"}, inplace=True)
df_final['Appraised Value'] = df_final['Appraised Value'].astype(str)
df_final['Appraised Value'] = df_final['Appraised Value'].apply(lambda x: ','.join(map(str, x)))  # converts list values to str
df_final['Appraised Value'] = df_final['Appraised Value'].apply(lambda x: x.replace(",", ""))
df_final = df_final.sort_values(by="Appraised Value", ascending=False)

df_final.to_csv('all_available_domains_appraised.csv')
files.download('all_available_domains_appraised.csv')
print("--------------------------------------------------------")
print(red("Exported appraised_domains_output.csv!"))

# Check Keyword Slope with PyTrends
H/t to Paul Shapiro for the idea, and a lot of the code! https://searchwilderness.com/google-trends-api-slope/

In [ ]:
# Set Google Trends Variable
time_var = 'today 5-y'  # 'today #-m'  # 'all'  #
geo_var = 'GB'
random_delay_max = 3  # max random delay in seconds - pyTrends

In [ ]:
# Start:: Keyword Slope ::
df_read_csv = df_final[['Keyword']] 
searchKeywords = df_read_csv['Keyword'].tolist()
# remove duplicates from list by creating a set then returning to a list
searchKeywords = set(searchKeywords)
searchKeywords = list(searchKeywords)

def chunks(items, chunkSize):
    for i in range(0, len(items), chunkSize):
        yield items[i:i + chunkSize]

list(chunks(searchKeywords, 5))

# delete old chunk .csv files
files = glob.glob('*chunk*.csv')
for f in files:
    os.remove(f)

print("--------------------------------------------------------")
print(magenta("Querying Google Trends via PyTrends"))
print("--------------------------------------------------------")
print("Look Back Period:", time_var)
print("Geo Location:", geo_var)
print("Random Delay: 1 -", random_delay_max, "seconds")
print("--------------------------------------------------------")

# putting it all together (copying in calls to pytrends library to calculate interest_over_time)
for chunkIndex, chunk in enumerate(chunks(searchKeywords, 5)):
    # print('%2d) getting google trends for %s...' % (chunkIndex+1, chunk), end='')
    print((chunk), end='')
    chunkOutputFile = "chunk%02d.csv" % (chunkIndex + 1)
    pytrend.build_payload(kw_list=chunk, timeframe=time_var, geo=geo_var)
    interest_over_time_df = pytrend.interest_over_time()
    interest_over_time_df.to_csv(path_or_buf=chunkOutputFile)
    print(' - done! Saved to %s' % chunkOutputFile)
    sleep(randint(1, random_delay_max))

# concatenate chunked parts into single df
all_files = glob.glob(
    os.path.join("*chunk*.csv")) 
df_from_each_file = (pd.read_csv(f) for f in all_files)

concatenated_df = pd.concat(df_from_each_file, axis=1)
concatenated_df_clean = (concatenated_df.drop('date', 1)).drop('isPartial', 1)

df_dates_file = pd.read_csv('chunk01.csv')
df_date_export = concatenated_df.iloc[:, 0]
final_result = pd.concat([df_date_export, concatenated_df_clean], axis=1)

# give me the current year and month
yearToday = datetime.now().strftime('%Y')
lastYear = (datetime.now() - relativedelta(years=1)).strftime('%Y')
prevYear = (datetime.now() - relativedelta(years=2)).strftime('%Y')

lastYearResults = final_result[(final_result['date'] > lastYear) & (final_result['date'] < yearToday)]
prevYearResults = final_result[(final_result['date'] > prevYear) & (final_result['date'] < lastYear)]
keywords_to_check = list(final_result)

last_year_mean = lastYearResults[keywords_to_check].mean()
prev_year_mean = prevYearResults[keywords_to_check].mean()

xlast = last_year_mean
xprev = prev_year_mean
ylast = int(lastYear)
yprev = int(prevYear)

def slope_formula(xlast, xprev, ylast, yprev):
    return (xlast - xprev) / (ylast - yprev)

keywordFinallist = ((slope_formula(xlast, xprev, ylast, yprev)))

df_keywordFinallist = pd.DataFrame(keywordFinallist)
df_keywordFinallist = df_keywordFinallist.reset_index()
df_keywordFinallist.columns = ['Keyword', 'Slope']

# combine the godaddy and gtrends dataframes
df_combined = pd.merge(df_final, df_keywordFinallist, on="Keyword", how="left")

# delete old chunk .csv files
files = glob.glob('*chunk*.csv')
for f in files:
    os.remove(f)
df_combined.round(2)

#export trend slope dataframe
print("--------------------------------------------------------")
print(red("Exported trends_slope.csv"))
df_combined.to_csv("trends_slope.csv")
files.download('trends_slope.csv')
print("--------------------------------------------------------")
print(red("Exported trends_slope.csv!"))

# Scrape Live SERPS Using ZenSERP API
Used to check the competition. How many Websites have the keywords in the page title. Low number = Lower Competition in the SERPs. 

In [ ]:
# Set ZEN SERP Variables
device_var = "Desktop"
search_engine_var = "google.co.uk"
location_var = "London,England,United Kingdom"
geo_location_Var = "GB"
homepage_language_var = "en"

# ZEN SERP API options
limit_api_searches = True  # use to limit the number of Zen SERP api calls
api_credits = 20  # if API Limited, how many credits?
sort_on = "Volume"  # "Volume", "Global volume", "Price", "Appraised Value", "Country", "CPC", "Clicks", "Return Rate", "Slope"

In [ ]:
# Start:: Zen SERP
if not limit_api_searches:
    api_credits = found_domains_len

if api_credits > found_domains_len:
    api_credits = found_domains_len

print("--------------------------------------------------------")
print(magenta(("Scraping the SERPs! This will use ")) + magenta((api_credits)) + magenta((" Credit(s)")))
print("--------------------------------------------------------")
print("Device:", device_var)
print("Search Engine:", search_engine_var)
print("Location:", location_var)
print("Geo Location:", geo_location_Var)
print("Language:", homepage_language_var)
print("--------------------------------------------------------")

df_combined = df_combined.sort_values(by=sort_on, ascending=False)
df_combined.drop_duplicates(subset=['Keyword'], keep="first", inplace=True)
search_terms = df_combined['Keyword'].tolist()  # dump search term queries to a list

# trim list based on specified credits
search_terms = search_terms[:api_credits]

# make empty list and dataframe to store the extracted data
df_final = pd.DataFrame(None)
url_list = []
title_list = []
query_list = []
progress_count = 0

for i in search_terms:
    headers = {"apikey": zenserp_key}
    params = (
        ("q", i),
        ("device", device_var),
        ("search_engine", search_engine_var),
        ("location", location_var),
        ("gl", geo_location_Var),
        ("hl", homepage_language_var),
        ("apikey", zenserp_key),
    )
    progress_count = progress_count +1
    print(i.strip(), "- Searched", progress_count, "of", found_domains_len)
    response = requests.get('https://app.zenserp.com/api/v2/search', headers=headers, params=params);

    # Get JSON Data
    d = response.json()
    json_str = json.dumps(d)  # dumps the json object into an element
    resp = json.loads(json_str)  # load the json to a string
    organic = (resp['organic'])

    # get the length of the list to iterate over in the loop
    list_len = len(organic)

    counter = 0
    while counter != list_len:
        access = (organic[counter])
        #print("")

        try:
            my_url = (access['url'])
            url_list.append(my_url)
        except Exception:
            url_list.append("MISSING")
            pass

        try:
            my_title = (access['title'])
            title_list.append(my_title)
        except Exception:
            title_list.append("MISSING")
            pass

        query = (resp['query'])
        q_access = (query['q'])
        query_list.append(q_access)
        counter = counter + 1

# add lists to dataframe columns
df_final['query'] = query_list
df_final['url'] = url_list
df_final['title'] = title_list

df_final["query clean"] = df_final["query"]
df_final["query clean"] = (df_final["query clean"].str.replace(r"[^a-zA-Z ]+", " ").str.strip())  # strip special chars
df_final["query clean"] = df_final["query clean"].str.lower()
df_final["query clean"] = df_final["query clean"].replace('\s+', ' ', regex=True)  # replace whitespace

# clean up the page title to test how many times the exact keyword is found
df_final["title"] = (df_final["title"].str.replace(r"[^a-zA-Z ]+", " ").str.strip())  # strip out all special chars
df_final["title"] = df_final["title"].str.lower()
df_final["title"] = df_final["title"].replace('\s+', ' ', regex=True)  # replace whitespace
df_final["Keyword in Title?"] = df_final.apply(lambda row: row["query clean"] in row["title"], axis=1)  # kw in title?

df_final = df_final[~df_final["url"].isin(["MISSING"])]
df_final["url_count"] = 1

df_grouped = (df_final.groupby("query").agg({"Keyword in Title?": "sum", "url_count": "sum"}).reset_index())
df_grouped['% of Sites with KW in Title'] = df_grouped['Keyword in Title?'] / df_grouped['url_count'] * 100

cols = "query", "url", "title", "% of Sites with KW in Title"
df_grouped = df_grouped.reindex(columns=cols)

# make the stats dataframe
df_final_stats = pd.merge(df_final, df_grouped, on="query")
del df_final_stats['url_y']
del df_final_stats['title_y']

# rename the columns
df_final_stats.rename(columns={"url_x": "url", "title_x": "title"}, inplace=True)

# pivot values (tall to wide - so each url has it's own column)
df_final_stats['idx'] = df_final_stats.groupby('query').cumcount() + 1
df_final_stats = df_final_stats.pivot_table(index=['query', '% of Sites with KW in Title'], columns='idx',
                                            values=['url'], aggfunc='first')
df_final_stats = df_final_stats.sort_index(axis=1, level=1)
df_final_stats.columns = [f'{x}_{y}' for x, y in df_final_stats.columns]
df_final_stats = df_final_stats.reset_index()

# make the final dataframe
df_final_output = pd.merge(df_combined, df_final_stats, left_on="Keyword", right_on="query", how="left")
del df_final_output['query']
df_final_output['% of Sites with KW in Title'] = df_final_output['% of Sites with KW in Title'].astype(float)
df_final_output.round(2)

# export the final dataframe
print("--------------------------------------------------------")
print(red("Exported finished_output.csv!"))
print("--------------------------------------------------------")
df_final_output.to_csv("finished_output.csv", index=False)
files.download('finished_output.csv')
print("--------------------------------------------------------")
print(red("Exported finished_output.csv!"))